In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import re
from ast import literal_eval
from string import punctuation

import pandas as pd
from regex import UNICODE, VERBOSE, compile
from segtok.tokenizer import word_tokenizer

from src.data.main_processing import (remove_html_tags, remove_punctuation,
                                      remove_urls)

In [3]:
def is_punctuated(df, text_col="TEXT"):
    def punctuated(text):
        for x in text:
            if x in punctuation:
                return True
        else:
            return False
    return df[text_col].map(punctuated)

In [5]:
df = pd.read_csv("../data/one_hot_tags_related_only.csv", index_col=0)

In [6]:
df.shape

(54600, 18)

In [7]:
df.head()

,TITLE,LIKES,TEXT,TAGS,COMMENTS,VIEWS,NICKNAME,TIME_GAP,TIMESTAMP,CURRENT_TIMESTAMP,политика,помощь,дети,девушки,отношения,рассказ,стихи,юмор
3,Хорошо быть бабой,245,"\nХорошо быть бабой - загрустила, сфоткала себ...","['Девушки', 'Счастье']",38,NaN,Viruso4ek,5 лет назад,2014-12-22T23:52:56+03:00,1.585448e+09,0,0,0,1,0,0,0,0
6,Рим(история циклична),3,"\nЛегионы, легионы \nопоясали страну\nМатерят ...","['Рим', 'История', 'Война', 'Стихи']",12,NaN,lokibrother,5 лет назад,2014-12-22T23:49:08+03:00,1.585448e+09,0,0,0,0,0,0,1,0
9,Языковой барьер.,73,"\n Семья, по долгу отцовской профессии, по...","['Язык', 'Украина', 'Юмор']",3,NaN,alekrasnodar,5 лет назад,2014-12-22T23:38:20+03:00,1.585448e+09,0,0,0,0,0,0,0,1
10,не надо стесняться,79,"\nА что плохого в онанизме? В конце концов, вы...","['Вуди Аллен', 'Юмор']",15,NaN,durdom0550,5 лет назад,2014-12-22T23:37:24+03:00,1.585448e+09,0,0,0,0,0,0,0,1
14,Не могу вспомнить название пластинки из детства.,4,"\nВсем привет, в детстве была у меня пластинка...","['Пластинка', 'Склероз', 'Помощь', 'Старое', '...",10,NaN,NiiBird,5 лет назад,2014-12-22T23:25:27+03:00,1.585448e+09,0,1,0,0,0,0,0,0


### Remove urls

In [8]:
df["TEXT"].str.len().mean()

1365.7167216117216

In [9]:
remove_urls(df)

In [10]:
df["TEXT"].str.len().mean()

1363.3041941391941

### Remove punctuation

In [11]:
df["TEXT_PUNCT"] = df["TEXT"].copy()
df["TITLE_PUNCT"] = df["TITLE"].copy()

In [12]:
remove_punctuation(df, "TEXT")

In [13]:
df["TEXT"].str.len().mean()

1358.3367582417582

In [14]:
df["TITLE"].str.len().mean()

30.755641025641026

In [15]:
remove_punctuation(df, 'TITLE')

In [16]:
df["TITLE"].str.len().mean()

30.47705128205128

### Apply lower case

In [17]:
df["TEXT"] = df['TEXT'].str.lower()
df['TEXT_PUNCT'] = df['TEXT_PUNCT'].str.lower()

### Fix list records with json

In [18]:
df["TAGS"]=df['TAGS'].map(literal_eval)

In [19]:
from functools import partial
part_dumps = partial(json.dumps, ensure_ascii=False)

In [20]:
df['TAGS']=df['TAGS'].map(part_dumps)

In [21]:
df.drop(columns=['VIEWS'], inplace=True)

### Save results

In [23]:
df.to_csv("../data/preprocessed.csv", index=False)

### Split rows by target columns on strict and fuzzy classes

In [24]:
target_tags = ['политика', 'помощь', 'дети', 'девушки', 'отношения', 'рассказ', 'стихи', 'юмор']

In [37]:
df = pd.read_csv("../data/preprocessed.csv")

In [39]:
df[target_tags].sum(axis=0)

политика     10074
помощь       13588
дети          5521
девушки       4459
отношения     3347
рассказ       5276
стихи         4795
юмор         10832
dtype: int64

In [40]:
df_fuzzy = df.loc[df[target_tags].sum(axis=1) > 1]

In [41]:
df_fuzzy[target_tags].sum(axis=0)

политика      633
помощь        417
дети          656
девушки      1196
отношения    1065
рассказ       512
стихи         438
юмор         1559
dtype: int64

In [42]:
df_strict = df.drop(index=df_fuzzy.index)

In [50]:
numered_tags = list(enumerate(sorted(target_tags)))

In [51]:
numered_tags

[(0, 'девушки'),
 (1, 'дети'),
 (2, 'отношения'),
 (3, 'политика'),
 (4, 'помощь'),
 (5, 'рассказ'),
 (6, 'стихи'),
 (7, 'юмор')]

In [54]:
df_strict['target'] = sum([df_strict[tag].map(lambda z: x if z else 0) for x, tag in numered_tags])

In [55]:
df_strict

,TITLE,LIKES,TEXT,TAGS,COMMENTS,NICKNAME,TIME_GAP,TIMESTAMP,CURRENT_TIMESTAMP,политика,помощь,дети,девушки,отношения,рассказ,стихи,юмор,TEXT_PUNCT,TITLE_PUNCT,target
0,Хорошо быть бабой,245,\nхорошо быть бабой загрустила сфоткала себ...,"[""Девушки"", ""Счастье""]",38,Viruso4ek,5 лет назад,2014-12-22T23:52:56+03:00,1.585448e+09,0,0,0,1,0,0,0,0,"\nхорошо быть бабой - загрустила, сфоткала себ...",Хорошо быть бабой,0
1,Рим история циклична,3,\nлегионы легионы \nопоясали страну\nматерят ...,"[""Рим"", ""История"", ""Война"", ""Стихи""]",12,lokibrother,5 лет назад,2014-12-22T23:49:08+03:00,1.585448e+09,0,0,0,0,0,0,1,0,"\nлегионы, легионы \nопоясали страну\nматерят ...",Рим(история циклична),6
2,Языковой барьер,73,\n семья по долгу отцовской профессии по...,"[""Язык"", ""Украина"", ""Юмор""]",3,alekrasnodar,5 лет назад,2014-12-22T23:38:20+03:00,1.585448e+09,0,0,0,0,0,0,0,1,"\n семья, по долгу отцовской профессии, по...",Языковой барьер.,7
3,не надо стесняться,79,\nа что плохого в онанизме в конце концов вы...,"[""Вуди Аллен"", ""Юмор""]",15,durdom0550,5 лет назад,2014-12-22T23:37:24+03:00,1.585448e+09,0,0,0,0,0,0,0,1,"\nа что плохого в онанизме? в конце концов, вы...",не надо стесняться,7
4,Не могу вспомнить название пластинки из детства,4,\nвсем привет в детстве была у меня пластинка...,"[""Пластинка"", ""Склероз"", ""Помощь"", ""Старое"", ""...",10,NiiBird,5 лет назад,2014-12-22T23:25:27+03:00,1.585448e+09,0,1,0,0,0,0,0,0,"\nвсем привет, в детстве была у меня пластинка...",Не могу вспомнить название пластинки из детства.,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54595,Вдруг откуда не возьмись,-70,\nеще совсем недавно все политологи и экспе...,"[""Украина"", ""Кавказ"", ""Ненависть"", ""Любовь"", ""...",7,niochemnegovory,4 года назад,2015-06-26T01:12:42+03:00,1.585572e+09,1,0,0,0,0,0,0,0,"\nеще совсем недавно все ""политологи"" и ""экспе...",Вдруг откуда не возьмись.....,3
54596,Совет будущим студентам,1771,\nсамое главное в общежитии сохранять хороши...,"[""Общежитие"", ""Девушки"", ""Студенты""]",72,samparam,4 года назад,2015-06-26T00:57:53+03:00,1.585572e+09,0,0,0,1,0,0,0,0,\nсамое главное в общежитии - сохранять хороши...,Совет будущим студентам,0
54597,Пара анекдотов,-12,\nк психоaнaлитику приходит нa прием женщинa ...,"[""Анекдот"", ""Юмор"", ""Веселье""]",1,5Aliment,4 года назад,2015-06-26T00:32:45+03:00,1.585572e+09,0,0,0,0,0,0,0,1,\nк психоaнaлитику приходит нa прием женщинa. ...,Пара анекдотов,7
54598,Правдивая история о том как Лена работала в п...,512,\nя с большим наслаждением читала рассказы пик...,"[""Длиннотекст"", ""Работа"", ""Детский дом"", ""Прию...",80,CheburashkinoUho,4 года назад,2015-06-26T00:20:05+03:00,1.585572e+09,0,0,1,0,0,0,0,0,\nя с большим наслаждением читала рассказы пик...,"Правдивая история о том, как Лена работала в п...",1


In [43]:
df_strict[target_tags].sum(axis=0)

политика      9441
помощь       13171
дети          4865
девушки       3263
отношения     2282
рассказ       4764
стихи         4357
юмор          9273
dtype: int64

In [56]:
for x in target_tags:
    del df_strict[x]

In [57]:
df_strict

,TITLE,LIKES,TEXT,TAGS,COMMENTS,NICKNAME,TIME_GAP,TIMESTAMP,CURRENT_TIMESTAMP,TEXT_PUNCT,TITLE_PUNCT,target
0,Хорошо быть бабой,245,\nхорошо быть бабой загрустила сфоткала себ...,"[""Девушки"", ""Счастье""]",38,Viruso4ek,5 лет назад,2014-12-22T23:52:56+03:00,1.585448e+09,"\nхорошо быть бабой - загрустила, сфоткала себ...",Хорошо быть бабой,0
1,Рим история циклична,3,\nлегионы легионы \nопоясали страну\nматерят ...,"[""Рим"", ""История"", ""Война"", ""Стихи""]",12,lokibrother,5 лет назад,2014-12-22T23:49:08+03:00,1.585448e+09,"\nлегионы, легионы \nопоясали страну\nматерят ...",Рим(история циклична),6
2,Языковой барьер,73,\n семья по долгу отцовской профессии по...,"[""Язык"", ""Украина"", ""Юмор""]",3,alekrasnodar,5 лет назад,2014-12-22T23:38:20+03:00,1.585448e+09,"\n семья, по долгу отцовской профессии, по...",Языковой барьер.,7
3,не надо стесняться,79,\nа что плохого в онанизме в конце концов вы...,"[""Вуди Аллен"", ""Юмор""]",15,durdom0550,5 лет назад,2014-12-22T23:37:24+03:00,1.585448e+09,"\nа что плохого в онанизме? в конце концов, вы...",не надо стесняться,7
4,Не могу вспомнить название пластинки из детства,4,\nвсем привет в детстве была у меня пластинка...,"[""Пластинка"", ""Склероз"", ""Помощь"", ""Старое"", ""...",10,NiiBird,5 лет назад,2014-12-22T23:25:27+03:00,1.585448e+09,"\nвсем привет, в детстве была у меня пластинка...",Не могу вспомнить название пластинки из детства.,4
...,...,...,...,...,...,...,...,...,...,...,...,...
54595,Вдруг откуда не возьмись,-70,\nеще совсем недавно все политологи и экспе...,"[""Украина"", ""Кавказ"", ""Ненависть"", ""Любовь"", ""...",7,niochemnegovory,4 года назад,2015-06-26T01:12:42+03:00,1.585572e+09,"\nеще совсем недавно все ""политологи"" и ""экспе...",Вдруг откуда не возьмись.....,3
54596,Совет будущим студентам,1771,\nсамое главное в общежитии сохранять хороши...,"[""Общежитие"", ""Девушки"", ""Студенты""]",72,samparam,4 года назад,2015-06-26T00:57:53+03:00,1.585572e+09,\nсамое главное в общежитии - сохранять хороши...,Совет будущим студентам,0
54597,Пара анекдотов,-12,\nк психоaнaлитику приходит нa прием женщинa ...,"[""Анекдот"", ""Юмор"", ""Веселье""]",1,5Aliment,4 года назад,2015-06-26T00:32:45+03:00,1.585572e+09,\nк психоaнaлитику приходит нa прием женщинa. ...,Пара анекдотов,7
54598,Правдивая история о том как Лена работала в п...,512,\nя с большим наслаждением читала рассказы пик...,"[""Длиннотекст"", ""Работа"", ""Детский дом"", ""Прию...",80,CheburashkinoUho,4 года назад,2015-06-26T00:20:05+03:00,1.585572e+09,\nя с большим наслаждением читала рассказы пик...,"Правдивая история о том, как Лена работала в п...",1


In [58]:
df_strict.to_csv('../data/preprocessed_strict.csv')

In [32]:
df_fuzzy.to_csv('../data/preprocessed_fuzzy.csv')

In [33]:
df_strict = pd.read_csv('../data/preprocessed_strict.csv')

In [34]:
df_strict.shape

(51416, 20)

In [35]:
df_strict.drop(index=df_strict[df_strict['TEXT'].isnull()].index, inplace=True)

In [36]:
df_strict.shape

(51416, 20)

In [ ]:
df_strict.to_csv('../data/processed/preprocessed_strict.csv')